# Pre-processing

## 1. Download video data

In [ ]:
import boto3, botocore
import os
import pandas as pd
import pickle
import pprint
from pathlib import Path
session = boto3.Session(
    aws_access_key_id='XXX',
    aws_secret_access_key='XXX')

s3 = session.resource('s3')

BUCKET = 'XXX'

In [55]:
prefix1 = 'VideoDataForAnalyses' # OR AnonymisedVideoDataForAnalyses
prefix2 = [str(n) for n in list(range(6,7,1))] # Participant ID

for i in range(len(prefix2)):
    targetPrefix = prefix1 + '/c' + prefix2[i].zfill(2) + '/'  # c OR t
    outPath = './data/' + prefix1 + '/c' + prefix2[i].zfill(2)      # c OR t
    
    os.makedirs(outPath, exist_ok=True)

    for object_summary in s3.Bucket(BUCKET).objects.filter(Prefix=targetPrefix):
        targetFilePath = outPath + '/' + object_summary.key.split('/')[2]
        print(object_summary.key)
        print(targetFilePath)
        s3.Bucket(BUCKET).download_file(object_summary.key, targetFilePath)
    
    print(object_summary.key.split('/')[1]+": DONE")

VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_01_1.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_01_1.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_01_2.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_01_2.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_02_1.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_02_1.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_02_2.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_02_2.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_03_1.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_03_1.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_03_2.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_03_2.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_04_1.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_04_1.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_04_2.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_04_2.mp4
VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_05_1.mp4
./VideoDataForAnalyses/c06/c06_Lc_D30_Sc00_Rc_05_1.mp4
VideoDataForAnalyses/c06/c06

## 2. Run iCatcher+

In [ ]:
# conda activate icatcher_plus
# cd ..
# cd icatcher_plus
# python /mnt/c/Users/intern_mbah/aws_video/_agc_analysis/run_icatcher.py
# conda deactivate
# cd ..
# cd aws_video/_agc_analysis

In [7]:
%pwd     # '/mnt/c/Users/intern_mbah/aws_video/_agc_analysis'

'/mnt/c/Users/intern_mbah/aws_video/_agc_analysis'

## 3. Make summary files (iCatcher+)

### 3.1. Participant level

In [5]:
import csv
import os
import pandas as pd
dir = 'VideoDataForAnalyses'   # OR AnonymisedVideoDataForAnalyses
input_folders = os.listdir('./res_icatcher/'+dir)
output_path = './res_icatcher_summary/'+dir
os.makedirs(output_path, exist_ok=True)

In [14]:
for target_folder in input_folders:
    input_files = os.listdir('./res_icatcher/'+dir+'/'+target_folder+'/text')
    res_list =[]

    i=0
    for target_file in input_files: 
        base = target_file.split('_')
        # read the text file
        with open('./res_icatcher/'+dir+'/'+target_folder+'/text/'+target_file) as f:
            res_raw = list(csv.reader(f))

        for res_line in res_raw:
            res_list.append(base + res_line + [target_file])

        i=i+1

    # Write to csv file
    res_list_pd = pd.DataFrame(res_list, columns=['id', 'lighting', 'distance', 'side', 'rotation', 'trial', 'time', 'frame', 'annotation', 'confidence', 'filename'])
    res_list_pd['time'] = res_list_pd['time'].str.strip('.txt')
    res_list_pd.to_csv(output_path+'/'+target_folder+'.csv', header=True, index=False)
    print(str(i)+': '+target_folder, ': Export to csv file, done')

540: c01 : Export to csv file, done
540: c02 : Export to csv file, done
540: c03 : Export to csv file, done
540: c04 : Export to csv file, done
540: c05 : Export to csv file, done
540: c06 : Export to csv file, done
540: c07 : Export to csv file, done
540: c08 : Export to csv file, done
540: c09 : Export to csv file, done
540: c10 : Export to csv file, done
540: c11 : Export to csv file, done
540: c12 : Export to csv file, done
540: c13 : Export to csv file, done
540: c14 : Export to csv file, done
540: c15 : Export to csv file, done
540: c16 : Export to csv file, done
540: c17 : Export to csv file, done
540: c18 : Export to csv file, done
540: c19 : Export to csv file, done
540: c20 : Export to csv file, done
540: c21 : Export to csv file, done
540: c22 : Export to csv file, done
540: c23 : Export to csv file, done
540: c24 : Export to csv file, done
540: c25 : Export to csv file, done
540: c26 : Export to csv file, done
540: c27 : Export to csv file, done
540: c28 : Export to csv fil

### 3.2. Dataset level

In [17]:
# Confirm the number of frames in each file
dir = 'dVideoDataForAnalyses'   # OR AnonymisedVideoDataForAnalyses
path = './res_icatcher_summary/'+dir
input_files = os.listdir(path)

last_frame_list = pd.DataFrame(columns=['id', 'lighting', 'distance', 'side', 'rotation', 'trial', 'time', 'frame', 'annotation', 'confidence', 'filename'])

for target_file in input_files:
    df = pd.read_csv(path+'/'+target_file)
    num_frame = df.drop_duplicates(subset=['filename'], keep='last', ignore_index=True)
    last_frame_list = pd.concat([last_frame_list, num_frame])

# The actual number of frames = last frame + 5 
last_frame_list['frame'] = last_frame_list['frame'] + 5
last_frame_list.to_csv(path+'/_last_frame.csv', header=True, index=False)
print('Number of frames: Mean: {0}, SD: {1}, Min: {2}, Max: {3}'.format(last_frame_list['frame'].mean(), last_frame_list['frame'].std(), last_frame_list['frame'].min(), last_frame_list['frame'].max()))

Number of frames: Mean: 27.8279012345679, SD: 6.213607593601454, Min: 17, Max: 33


In [ ]:
# You can also try this for a particular folder:
# python ffprobe.py

In [18]:
num_sample = 1 # down sampling
i = 1 # seed (set 1 for the main analysis, 97 for confirmation of the results& consistency)
sample_list = pd.DataFrame(columns=['id', 'lighting', 'distance', 'side', 'rotation', 'trial', 'time', 'frame', 'annotation', 'confidence', 'filename'])

for target_file in input_files:
    df = pd.read_csv(path+'/'+target_file)
    condnames = df.drop_duplicates(subset=['filename'], keep='last')['filename']
    condnames = condnames.values.tolist()
    for target_cond in condnames:
        tmp = df[df['filename'] == target_cond]
        tmp = tmp.sample(n=num_sample, random_state=i, replace=False, ignore_index=True)
        sample_list = pd.concat([sample_list, tmp])
        i = i+1
    print('Target file: {0}, i: {1}'.format(target_file, i))

sample_list.to_csv(path+'/_data_sampled.csv', header=True, index=False)
print('Number of participants down sampled: '+str(int(i/540)))

Target file: c01.csv, i: 637
Target file: c02.csv, i: 1177
Target file: c03.csv, i: 1717
Target file: c04.csv, i: 2257
Target file: c05.csv, i: 2797
Target file: c06.csv, i: 3337
Target file: c07.csv, i: 3877
Target file: c08.csv, i: 4417
Target file: c09.csv, i: 4957
Target file: c10.csv, i: 5497
Target file: c11.csv, i: 6037
Target file: c12.csv, i: 6577
Target file: c13.csv, i: 7117
Target file: c14.csv, i: 7657
Target file: c15.csv, i: 8197
Target file: c16.csv, i: 8737
Target file: c17.csv, i: 9277
Target file: c18.csv, i: 9817
Target file: c19.csv, i: 10357
Target file: c20.csv, i: 10897
Target file: c21.csv, i: 11437
Target file: c22.csv, i: 11977
Target file: c23.csv, i: 12517
Target file: c24.csv, i: 13057
Target file: c25.csv, i: 13597
Target file: c26.csv, i: 14137
Target file: c27.csv, i: 14677
Target file: c28.csv, i: 15217
Target file: c29.csv, i: 15757
Target file: c30.csv, i: 16297
Target file: t01.csv, i: 16837
Target file: t02.csv, i: 17377
Target file: t03.csv, i: 17

In [19]:
i/60

541.6166666666667